In [8]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
import pandas as pd
import visuals as vs
#Refer: http://scikit-learn.org/stable/modules/multiclass.html#multioutput-classification

In [9]:
features = pd.read_csv("F:\\ML\\MultiTarget\\features.csv")
target   = pd.read_csv("F:\\ML\\MultiTarget\\targets.csv")

In [20]:
print(features.shape, target.shape)
features.tail()

(1087, 102) (1087, 14)


,Set,Profile,expressivity,class_count,indiv_count,obj_property_count,dataproperty_count,datatypes_count,Cyclic Cls,HC_Cohesion,...,ReflexiveObjectProperty,InverseFunctionalObjectProperty,EquivalentDataProperties,SameIndividual,EquivalentObjectProperties,SubDataPropertyOf,FunctionalDataProperty,IrrefexiveObjectProperty,AsymmetricObjectProperty,DisjointObjectProperties
1082,Test,DL,SHIQ(D),1851,120,89,4,3,0.000,0.009,...,0.0,0.000,0.0,0.0,0.000,0.000,0.105,0.0,0.0,0.0
1083,Test,DL,SROIQ(D),697,3795,176,14,6,0.000,0.016,...,0.0,0.000,0.0,0.0,0.017,0.000,0.062,0.0,0.0,0.0
1084,Test,DL,ALCROIQ(D),389,39,152,51,7,2.222,0.016,...,0.0,0.178,0.0,0.0,0.000,2.844,1.867,0.0,0.0,0.0
1085,Test,EL,ALE+,2314,0,6,0,0,0.000,0.003,...,0.0,0.000,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.0
1086,Test,DL,SHOIQ(D),1382,59,76,21,5,0.208,0.014,...,0.0,0.035,0.0,0.0,0.000,0.351,0.280,0.0,0.0,0.0


In [22]:
def df_lbl_enc(df):
    for c in df.columns:
        if df[c].dtype == 'object':
            lbl = preprocessing.LabelEncoder()
            df[c] = lbl.fit_transform(df[c])
            print(c)
    return df

train_features = features.loc[features['Set'] == "Train"] 
test_features = features.loc[features['Set'] == "Test"] 
train_target  = target.loc[target['Set'] == "Train"] 
test_target   = target.loc[target['Set'] == "Test"]

cols = [c for c in features if c not in ['Set','Profile']]
cols = [c for c in features if c not in ['Set','Profile']]

train_features = train_features[cols]
test_features  = test_features[cols]
train_target   = train_target[cols]
test_target    = test_target[cols]

train_target.head()
#test_cols = [c for c in test_features if c not in ['Set','Profile']]

KeyError: "['expressivity' 'class_count' 'indiv_count' 'obj_property_count'\n 'dataproperty_count' 'datatypes_count' 'Cyclic Cls' 'HC_Cohesion'\n 'HC_MAX_DEPTH' 'HC_Max_Sibling' 'HC_Avg_Sibling' 'C_Tangledness'\n 'C_MaxTangledness' 'HP_Cohesion' 'P_Tangledness' 'P_MaxTangledness'\n 'HP_MAX_DEPTH' 'HP_Max_Sibling' 'HP_Avg_Sibling' 'OP_Cohesion'\n 'Onto_Cohesion' 'RRichness' 'AttRichness' 'UI' 'EUR' 'Axioms_count'\n 'AxiomsLogical_count' 'Tbox_size' 'Abox_size' 'Rbox_size' 'Tbox_ratio'\n 'Abox_ratio' 'Rbox_ratio' 'GCI' 'PCD' 'NPCD' 'GCI_Ratio' 'PCD_Ratio'\n 'NPCD_Ratio' 'MaxNesting' 'AvgNesting' 'ClsDisjoinedRatio'\n 'ClsWithNominalsRatio' 'IndivSameRatio' 'IndivDiffRatio'\n 'NominalsTBoxRatio' 'MaxACC' 'OntoCCDensity' 'Intersection' 'Union'\n 'Existentiel' 'Universal' 'LeastCardinality' 'MostCardinality'\n 'ExactCardinality' 'avgValuesCard' 'LogestMCard' 'LogestLCard'\n 'LogestECard' 'Complement' 'OneOf' 'HasValue' 'HasSelf' 'InversePropRatio'\n 'TransitivePropRatio' 'ReflexivePropRatio